In [1]:
import pandas as pd

# Data Cleaning Script for CHRP-AI

### Get data to be cleaned

In [121]:
# filepath to data to be cleaned
filepath = '/Users/ethanweiss/Desktop/Class documents/Spring 2020/Senior Design/Chiller Tons/Load_Data_Full.csv'
# read data from csv into pandas dataframe
data = pd.read_csv(filepath)

In [122]:
# initialize dictionary for unreliable indecies
unreliable_dict = {}

### Get index of UNRELIABLE points
##### Store indecies of unreliable data points in dictionary where keys are column indecies

In [123]:
# get indecies for all "eDNA Status as String" columns
status_cols = [i for i in range(len(data.columns)) if i%3 == 2]
# chiller_cols = [i for i in range(len(data.columns)) if i%3 == 0]
# chiller_col_labels = [data.columns[x] for x in chiller_cols]

# Loop through all status columns 
for j in status_cols:
    unr_index = [] # reset unreliable index list to empty
    for i in range(len(data)): # loop through all rows of data
        if data.iloc[i,j] == 'UNRELIABLE': # check if unreliable
            unr_index.append(i) # if unreliable, add index to list
    unreliable_dict[j] = unr_index # assign list of unreliable indecies to column index key

In [124]:
# DEBUG
# Count number of UNRELIABLE points
total = 0
for x in unreliable_dict.keys():
    total += len(unreliable_dict[x])
print(total)

5151


### **Use Linear Interpolation to get values for previously UNRELIABLE data points**
Assuming that UNRELIABLE data points occur in bunches, we want to linearly interpolate using the boundary values of Reliable data points. The "before bound" will be the value of the Reliable data point which occured right before the bunch of UNRELIABLE data points. The "after bound" will be the value of the Reliable data point which occurs right after the bunch of UNRELIABLE data points.

Process for Linear Interpolation: <br>
    1) Find "before index" --> Index of previous Reliable data point <br>
    2) Find "before bound" --> Value of previous Reliable data point <br>
    3) Find "after index" --> Index of next Reliable data point <br>
    4) Find "after bound" --> Value of next Reliable data point <br>
    5) Calculate slope --> $slope = \frac{a_{bound}-b_{bound}}{a_{index}-b_{index}}$ <br>
    6) Calculate new value --> $b_{bound} + slope * (c_{index} - b_{index})$ <br> 


In [125]:
# loop through all column indecies with unreliable data
for x in unreliable_dict.keys():
    # loop through all unreliable values
    for i in unreliable_dict[x]:
        # loop through all indecies before unreliable point
        for j in range(1,i):
            if data.iloc[i-j,x] != 'UNRELIABLE': # check for unreliable point
                # IF NOT UNRELIABLE: 
                before_bound = data.iloc[i-j,x-1] # set before bound value
                before_index = i-j # set before index value
                break # break loop once before bound is found
        # loop through all indecies after unreliable point
        for j in range(1,len(data) - i): 
            if data.iloc[i+j,x] != 'UNRELIABLE': # check for unreliable point
                # IF NOT UNRELIABLE:
                after_bound = data.iloc[i+j,x-1] # set after bound value
                after_index = i+j # set after index value
                break # break loop once after bound is found
       
        # Linear Interpolation to substitute for UNRELIABLE values
        slope = (after_bound - before_bound)/(after_index - before_index)
        data.iloc[i,x-1] = before_bound + slope * (i - before_index)

# Replace 'UNRELIABLE' tags with 'LI' tags
for x in unreliable_dict.keys():
    for i in unreliable_dict[x]:
        data.iloc[i,x] = 'LI'
        
        # DEBUG
#         if (x == 32 and i == 354):
#             print('before bound: {0}'.format(before_bound))
#             print('before index: {0}'.format(before_index))
#             print('after bound: {0}'.format(after_bound))
#             print('after index: {0}'.format(after_index))
#             print('slope: {0}'.format(slope))

In [128]:
# DEBUG
# Count number of LI tags
total = 0
for j in status_cols:
    for i in range(len(data)):
        if data.iloc[i,j] == 'LI':
            total += 1
print(total)

5151


### Save dataset with 'LI' tags as CSV

In [132]:
filepath = '/Users/ethanweiss/Desktop/Class documents/Spring 2020/Senior Design/Chiller Tons/Load_Data_Full_LItags.csv'
data.to_csv(filepath)

### Only keep relevant columns and rename accordingly
Keep only the first Date/time column and all Values columns. Rename Values columns according to chiller name.

In [142]:
# Get chiller names
chiller_cols = [i for i in range(len(data.columns)) if i%3 == 0]
chiller_cols = [data.columns[x] for x in chiller_cols]
# Get value columns to keep
value_cols = [i for i in range(len(data.columns)) if i%3 == 1]
value_cols = [data.columns[x] for x in value_cols]
# Compile columns to keep in dataframe
data.rename({'AHSC_CHL01_TONS':'Date/Time'},axis = 1,inplace = True)
keep_cols = ['Date/Time'] + value_cols
# Create Dictionary for guide to rename columns with appropriate chiller names
rename_dict = {}
for i in range(len(keep_cols[1:])):
    rename_dict[keep_cols[i]] = chiller_cols[i]
# Assign new column names (appropriate chiller names)
rel_data.rename(rename_dict,axis = 1,inplace = True)

### Save CSV of relevant data

In [205]:
filepath = '/Users/ethanweiss/Desktop/Class documents/Spring 2020/Senior Design/Chiller Tons/Load_Data_Full_ChillerValuesOnly.csv'
rel_data.to_csv(filepath)

In [206]:
rel_data.head()

,Date/Time,AHSC_CHL01_TONS,AHSC_CHL02_TONS,AHSC_CHL03_TONS,CHRP_CHL08_TONS,CHRP_CHL09_TONS,CHRP_CHL06_TONS,CHRP_CHL07_TONS,CHRP_CHL10_TONS,CHRP_CHL11_TONS,...,CRB_CHL07_TONS,CRB_CHL08_TONS,CRB_CHL09_TONS,CRB_CHL10_TONS,CRB_CHL01B_TONS,AHSC_CHL04_TONS,AHSC_CHL07_TONS,AHSC_CHL08_TONS,CRB_CHL02B_TONS,CRB_CHL03B_TONS
0,2019-02-12 00:00:00,1208.309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2181.195,...,893.11150,0.0,0.0,0.0,0.0,1047.702,0.0,0.0,0.0,0.0
1,2019-02-12 00:02:00,1211.706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2176.634,...,883.93870,0.0,0.0,0.0,0.0,1057.453,0.0,0.0,0.0,0.0
2,2019-02-12 00:04:00,1212.154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2162.145,...,886.16180,0.0,0.0,0.0,0.0,1058.788,0.0,0.0,0.0,0.0
3,2019-02-12 00:06:00,1208.656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2163.059,...,881.93895,0.0,0.0,0.0,0.0,1060.142,0.0,0.0,0.0,0.0
4,2019-02-12 00:08:00,1211.696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2163.059,...,877.71610,0.0,0.0,0.0,0.0,1062.337,0.0,0.0,0.0,0.0


### Find Outliers
Use 1 hour interval (15 points before and 15 points after current point)

For points with only N points before (w/ N < 15), use N points before and 30-N points after.<br>
For points with only N points after (w/ N < 15), use N points after and 30-N points before.

In [ ]:
outlier_cols = []
for x in rel_data.columns[1:]:
    outlier_col = [0 for i in range(len(rel_data))]
    for i in range(len(rel_data)):
        if i < 15:
            window = rel_data.loc[0:29,x]
            q1 = window.quantile(0.25)
            q3 = window.quantile(0.75)
            iqr = abs(q3 - q1)
            if ((rel_data.loc[i,x] < q1 - 1.5 * iqr) or (rel_data.loc[i,x] > q3 + 1.5 * iqr)):
                outlier_col[i] = 1
        elif i > len(rel_data) - 15:
            window = rel_data.loc[len(rel_data) - 30:len(rel_data) - 1,x]
            q1 = window.quantile(0.25)
            q3 = window.quantile(0.75)
            iqr = abs(q3 - q1)
            if ((rel_data.loc[i,x] < q1 - 1.5 * iqr) or (rel_data.loc[i,x] > q3 + 1.5 * iqr)):
                outlier_col[i] = 1
        else:
            window = rel_data.loc[i-15:i+14,x]
            q1 = window.quantile(0.25)
            q3 = window.quantile(0.75)
            iqr = abs(q3 - q1)
            if ((rel_data.loc[i,x] < q1 - 1.5 * iqr) or (rel_data.loc[i,x] > q3 + 1.5 * iqr)):
                outlier_col[i] = 1
    outlier_cols.append(outlier_col)

In [204]:
rel_data.columns

Index(['Date/Time', 'AHSC_CHL01_TONS', 'AHSC_CHL02_TONS', 'AHSC_CHL03_TONS',
       'CHRP_CHL08_TONS', 'CHRP_CHL09_TONS', 'CHRP_CHL06_TONS',
       'CHRP_CHL07_TONS', 'CHRP_CHL10_TONS', 'CHRP_CHL11_TONS',
       'CHRP_CHL12_TONS', 'CRB_CHL05_TONS', 'CRB_CHL06_TONS', 'CRB_CHL07_TONS',
       'CRB_CHL08_TONS', 'CRB_CHL09_TONS', 'CRB_CHL10_TONS', 'CRB_CHL01B_TONS',
       'AHSC_CHL04_TONS', 'AHSC_CHL07_TONS', 'AHSC_CHL08_TONS',
       'CRB_CHL02B_TONS', 'CRB_CHL03B_TONS'],
      dtype='object')